In [1]:
import yfinance as yf
import requests
import numpy as np
import pandas as pd
import ipynb

In [9]:
import nbimporter
from Financial_Data import search_stock_symbol,symbol_changes,pe_and_market_cap,rsi,analyze_rsi,macd,interpret_macd,fibonacci_retracement,fib_insight,calculate_trend,analyze_trend

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
def calculate_signal(symbol, data, current_date):
    # Slice the data up to the current date
    current_data = data.loc[:current_date]
    
    # Get the latest info
    comp = yf.Ticker(symbol)
    info = comp.info
    
    curr_price = current_data['Close'].iloc[-1]
    
    weights = {
    'pe': 0.5,
    'rsi': 0.8,
    'macd': 0.8,
    'fib': 0.6,
    'trend': 0.6
}
    
    # PE RATIO AND CAP ID
    market_cap, pe, is_large_cap = pe_and_market_cap(info)
    
    if not is_large_cap:
        return 0  
    
    if pe <= 20:
        pe_score = 1  # Buy
    else:
        pe_score = -1  # Sell
    
    # RSI
    r, window = rsi(current_data)
    rsi_score, _ = analyze_rsi(r)
    
    # MACD
    ma = macd(current_data)
    macd_score, _ = interpret_macd(ma)
    
    # Fibonacci Retracement
    fib = fibonacci_retracement(current_data)
    current_price = current_data['Close'].iloc[-1]
    prev_price = current_data['Close'].iloc[-2]
    trend = fib['trend']
    levels = {
        'retracement': fib['retracement_levels'],
        'extension': fib['extension_levels']
    }
    _, fib_score = fib_insight(current_price=current_price, trend=trend, levels=levels,previous_price=prev_price)
    
    data_trend = calculate_trend(data, short_window=14, long_window=50)
    trend_score, _ = analyze_trend(data_trend)
    
    final_score = (
    (pe_score * weights['pe']) + 
    (rsi_score * weights['rsi']) + 
    (macd_score * weights['macd']) + 
    (fib_score * weights['fib']) +
    (trend_score * weights['trend']))
    
    if final_score > -0.3 and final_score < 0.3:
        ans = 0
        
    elif final_score >= 0.3:
        ans = 1
        
    elif final_score <= -0.3:
        ans = -1
    
    return ans

In [11]:
def backtest(symbol, initial_balance=10000, holding_period=5, stop_loss=-5, take_profit=10, future_window=5):
    # Download data
    stock = yf.Ticker(symbol)
    data = stock.history(period='2y', interval='1d')
    
    # Initialize variables
    position = 0
    balance = initial_balance
    shares = 0
    trades = []
    good_predictions = 0
    bad_predictions = 0
    
    # Iterate through each day
    for i, current_date in enumerate(data.index[:-future_window]):
        current_price = data.loc[current_date, 'Close']
        
        if len(data.loc[:current_date]) < 60:  # Skip the first 60 days to have enough data for indicators
            continue
        
        # Get the signal
        signal = calculate_signal(symbol, data, current_date)
        
        # Check if we need to sell due to stop loss, take profit, or holding period
        if position == 1:
            days_held = (current_date - trade_entry_date).days
            price_change = (current_price - entry_price) / entry_price * 100
            
            if price_change <= stop_loss or price_change >= take_profit or days_held >= holding_period:
                # Sell
                balance += shares * current_price
                trade_type = 'sell'
                trades.append((trade_type, current_date, current_price, shares, balance))
                
                future_price = data['Close'].iloc[i + future_window]
                if (signal == 1 and future_price > current_price) or (signal == -1 and future_price < current_price):
                    good_predictions += 1
                else:
                    bad_predictions += 1
                
                shares = 0
                position = 0
        
        # Execute trades based on signal
        if signal == 1 and position == 0:  # Buy signal
            shares_to_buy = balance // current_price
            cost = shares_to_buy * current_price
            balance -= cost
            shares += shares_to_buy
            position = 1
            trade_type = 'buy'
            trades.append((trade_type, current_date, current_price, shares_to_buy, balance))
            entry_price = current_price
            trade_entry_date = current_date
            
            # Check if the buy prediction was good
            future_price = data['Close'].iloc[i + future_window]
            if future_price > current_price:
                good_predictions += 1
            else:
                bad_predictions += 1
            
        elif signal == -1 and position == 1:  # Sell signal
            balance += shares * current_price
            trade_type = 'sell'
            trades.append((trade_type, current_date, current_price, shares, balance))
            
            # Check if the sell prediction was good
            future_price = data['Close'].iloc[i + future_window]
            if future_price < current_price:
                good_predictions += 1
            else:
                bad_predictions += 1
            
            shares = 0
            position = 0
    
    # Final value
    final_balance = balance + shares * data['Close'].iloc[-1]
    return_pct = (final_balance - initial_balance) / initial_balance * 100
    
    total_predictions = good_predictions + bad_predictions
    prediction_accuracy = (good_predictions / total_predictions * 100) if total_predictions > 0 else 0
    
    return {
        'final_balance': final_balance,
        'return_pct': return_pct,
        'trades': trades,
        'good_predictions': good_predictions,
        'bad_predictions': bad_predictions,
        'prediction_accuracy': prediction_accuracy
    }

In [ ]:
symbols = ['RELIANCE.NS','TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS','SBIN.NS', 'LICI.NS', 'ITC.NS'] 
results = {}
for symbol in symbols:
    result = backtest(symbol, holding_period=26, future_window=26)
    results[symbol] = result

# Analyze results
for symbol, result in results.items():
    print(f"\nResults for {symbol}:")
    print(f"Final Balance: Rs{result['final_balance']:.2f}")
    print(f"Return: {result['return_pct']:.2f}%")
    print(f"Number of trades: {len(result['trades'])}")
    print(f"Good trades: {result['good_predictions']}")
    print(f"Bad trades: {result['bad_predictions']}")
    print(f"Accuracy: {result['prediction_accuracy']:.2f}%")

In [13]:
for trade in results['RELIANCE.NS']['trades']:
            print(f"{trade[0].capitalize()} on {trade[1]}: Price Rs {trade[2]:.2f}, Shares: {trade[3]}, Balance: {round(trade[4],2)}")

Buy on 2023-01-09 00:00:00+05:30: Price Rs 2380.33, Shares: 4.0, Balance: 478.68
Sell on 2023-01-11 00:00:00+05:30: Price Rs 2315.57, Shares: 4.0, Balance: 9740.96
Buy on 2023-02-09 00:00:00+05:30: Price Rs 2159.65, Shares: 4.0, Balance: 1102.36
Sell on 2023-02-22 00:00:00+05:30: Price Rs 2180.59, Shares: 4.0, Balance: 9824.74
Buy on 2023-02-24 00:00:00+05:30: Price Rs 2184.99, Shares: 4.0, Balance: 1084.76
Sell on 2023-02-27 00:00:00+05:30: Price Rs 2170.47, Shares: 4.0, Balance: 9766.62
Buy on 2023-03-03 00:00:00+05:30: Price Rs 2186.55, Shares: 4.0, Balance: 1020.42
Sell on 2023-03-09 00:00:00+05:30: Price Rs 2162.58, Shares: 4.0, Balance: 9670.74
Buy on 2023-03-31 00:00:00+05:30: Price Rs 2136.73, Shares: 4.0, Balance: 1123.81
Sell on 2023-04-26 00:00:00+05:30: Price Rs 2165.19, Shares: 4.0, Balance: 9784.59
Buy on 2023-04-27 00:00:00+05:30: Price Rs 2178.90, Shares: 4.0, Balance: 1069.0
Sell on 2023-05-16 00:00:00+05:30: Price Rs 2249.25, Shares: 4.0, Balance: 10066.0
Buy on 2023-

In [14]:
avg26 = 0
for i in symbols:
    avg26 += results[i]['prediction_accuracy']

avg26 /= len(symbols)
avg26

50.0